<a href="https://colab.research.google.com/github/bonofaber/Low-Latency-Android-iOS-Linux-Windows-tvOS-macOS-Interactive-Audio-Platform/blob/master/copy_of_copy_of_copy_of_copy_of_scratchpad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv('https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv')
#df = pd.read_csv('https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv')
#print(df.info())
#print(df.describe())
print(df.head(3))

# 타이타닉 데이터 정제 - 결측치 대체
age_mean = df['Age'].mean()
df['Age'].fillna(age_mean, inplace = True)
emb_mode = df['Embarked'].mode()[0]
df['Embarked'].fillna(emb_mode, inplace = True)
df = df.drop('Cabin', axis = 1)  # 생략

# 독립 변수 축소, 생성
df['Family'] = df['SibSp'] + df['Parch']
df.info()

   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         8

In [ ]:
#정규화
# iris 데이터 정규화
"""
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
f = ['sepal.length','sepal.width','petal.length','petal.width']
for i in f:
  df[[i]]  = scaler.fit_transform(X[[i]])
print(df.head()) """

"\nfrom sklearn.preprocessing import MinMaxScaler\nscaler = MinMaxScaler()\nf = ['sepal.length','sepal.width','petal.length','petal.width']\nfor i in f:\n  df[[i]]  = scaler.fit_transform(X[[i]])\nprint(df.head()) "

In [ ]:
# 문자 String을 숫자로 변환

# Titanic : 라벨 인코딩
"""
from sklearn.preprocessing import LabelEncoder
LE = LabelEncoder()
#df1['Sex'] =  LE.fit_transform(df1['Sex'])
#df1['Sex'].replace({'female': 0, "male" : 1}, inplace = True)
df1.replace(['female', "male"], [0, 1], inplace = True)
df1['Embarked'] = LE.fit_transform(df1['Embarked'])
df1['Sex'].head()  """

# Titanic : 원핫 인코딩
oh_sex = pd.get_dummies(df['Sex'])
oh_embk = pd.get_dummies(df['Embarked'])
df1 = pd.concat([df, oh_sex, oh_embk], axis = 1)
print(df1.info())
print(df1.head(2))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 17 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          891 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Embarked     891 non-null    object 
 11  Family       891 non-null    int64  
 12  female       891 non-null    uint8  
 13  male         891 non-null    uint8  
 14  C            891 non-null    uint8  
 15  Q            891 non-null    uint8  
 16  S            891 non-null    uint8  
dtypes: float64(2), int64(6), object(4), uint8(5)
memory usage: 88.0+ KB
None
   PassengerId  Sur

In [ ]:
# 독립변수 종속변수 데이터 선택
# iris 데이터 선택
#X = df[['sepal.length','sepal.width','petal.length','petal.width']]
#y = df['variety']
#print(X.head(3))
#print('\n', y.head(3))

#Titanic 데이터 선택
#X = df[['Pclass', 'Sex', 'Age', 'Fare', 'Embarked', 'Family']]
X = df1[['Pclass', 'Age', 'Fare', 'Family', 'female', 'male', 'C', 'Q', 'S']] # 원핫 엔코딩 경우
y = df1['Survived'] 
print(X.head(3))
print('\n', y.head(3))

   Pclass   Age     Fare  Family  female  male  C  Q  S
0       3  22.0   7.2500       1       0     1  0  0  1
1       1  38.0  71.2833       1       1     0  1  0  0
2       3  26.0   7.9250       0       1     0  0  0  1

 0    0
1    1
2    1
Name: Survived, dtype: int64


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 11)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(712, 9) (179, 9) (712,) (179,)


In [ ]:
# 모델 선택 및 학습 검정
# 분류 모델 선택 - KNN K최근접 이윳
#from sklearn.neighbors import KNeighborsClassifier
#model = KNeighborsClassifier(n_neighbors = 5) # metric = 'euclidean' 선택적

# 분류 모델 선택 - 의사결정나무 
#from sklearn.tree import DecisionTreeClassifier
#model = DecisionTreeClassifier(random_state = 11) #, max_depth = 3) #선택

#분류모델 - svm,SVC
from sklearn.svm import SVC
model = SVC(kernel = 'rbf') # 옵션 'linear', C=1, gamma = 0.1)
model.fit(X_train, y_train)     # 트레인 데이터로 학습
pred = model.predict(X_test)    # 테스트 데이터로 예측 분류
#print(pred[:5])

# 성능평가 - 정확도 측정
from sklearn.metrics import accuracy_score
acc = accuracy_score(y_test, pred)   
print(acc)      #n_neighbors = 3, 6, 9로 변경하면서 측정

# 성능평가 - 오차행렬
from sklearn.metrics import confusion_matrix
mat = confusion_matrix(y_test,pred)
print(mat)

# 성능평가 - 분류보고서 classification_report() 이용한 평가 지표 계산
from sklearn.metrics import classification_report
rpt = classification_report(y_test,pred)
print(rpt)


0.7206703910614525
[[110   8]
 [ 42  19]]
              precision    recall  f1-score   support

           0       0.72      0.93      0.81       118
           1       0.70      0.31      0.43        61

    accuracy                           0.72       179
   macro avg       0.71      0.62      0.62       179
weighted avg       0.72      0.72      0.68       179



In [ ]:
# 결과를 화일로 저장 및 확인
#print(pred[:5])
#print(y_test[:5])
result = pd.DataFrame({'Real':y_test, 'Predicted':pred})
#print(result.head())
result.to_csv('result000.csv')#, index = False)
conf = pd.read_csv('result000.csv', index_col = 'Unnamed: 0')
print(conf.head())


     Real  Predicted
431     1          0
821     1          0
629     0          0
626     0          0
665     0          1
